# Inference Pipeline

TODO

### Imports

In [1]:
import pandas as pd
import hopsworks
import joblib
import datetime
from PIL import Image
from datetime import datetime
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot
from keras.utils import pad_sequences
import requests

from newsapi import NewsApiClient

### Connect to Hopsworks

In [3]:
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5322
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


### Get batch data from Hopsworks

In [ ]:
# Call hopsworks to get data
feature_view = fs.get_feature_view(name="headlines_sentiment_fg", version=1)
batch_data = feature_view.get_batch_data()

## Load Encoder

In [14]:
tokenizer = mr.get_model("headlines_sentiment_encoder", version=1)
model_dir = tokenizer.download()
tokenizer = joblib.load(model_dir + "/headlines_sentiment_encoder.pkl")

max_len = 60

In [10]:
print(tokenizer.get_config())

{'num_words': 5000, 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 'lower': True, 'split': ' ', 'char_level': False, 'oov_token': None, 'document_count': 11499, 'word_counts': '{"accord": 138, "gran": 2, "compani": 986, "plan": 161, "move": 70, "product": 329, "russia": 96, "although": 3, "grow": 41, "technopoli": 17, "develop": 236, "stage": 25, "area": 128, "less": 19, "100": 83, "000": 375, "squar": 25, "meter": 7, "order": 222, "host": 20, "work": 123, "comput": 16, "technolog": 165, "telecommun": 20, "statement": 34, "said": 563, "intern": 65, "electron": 73, "industri": 164, "elcoteq": 39, "laid": 10, "ten": 17, "employe": 128, "tallinn": 14, "facil": 85, "contrari": 2, "earlier": 88, "layoff": 10, "contract": 186, "rank": 13, "offic": 137, "worker": 137, "daili": 59, "postime": 1, "report": 469, "new": 744, "plant": 150, "would": 131, "increas": 241, "capac": 54, "meet": 98, "expect": 159, "demand": 56, "improv": 73, "use": 207, "raw": 23, "materi": 52, "therefor": 7, "profi

In [13]:
# TEST

X = tokenizer.texts_to_sequences(['test neg covid 19 union minist home quarantin'])
print(X)
X = pad_sequences(X, padding='post', maxlen = max_len)
print(X)

[[28, 740, 6, 5, 214, 224, 109, 163]]
[[ 28 740   6   5 214 224 109 163   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]]


### Load LSTM Model & Predict 

In [ ]:
model = mr.get_model("headlines_sentiment_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/headlines_sentiment_model.pkl")

In [ ]:
y_pred = model.predict(batch_data)
print(y_pred)

# Upload prediction TODO
dataset_api = project.get_dataset_api()    
dataset_api.upload("./latest_iris.png", "Resources/images", overwrite=True)

iris_fg = fs.get_feature_group(name="iris_modal", version=1)
df = iris_fg.read() 
#print(df)
label = df.iloc[-offset]["variety"]
label_url = "https://raw.githubusercontent.com/featurestoreorg/serverless-ml-course/main/src/01-module/assets/" + label + ".png"
print("Flower actual: " + label)
img = Image.open(requests.get(label_url, stream=True).raw)            
img.save("./actual_iris.png")
dataset_api.upload("./actual_iris.png", "Resources/images", overwrite=True)

monitor_fg = fs.get_or_create_feature_group(name="iris_predictions",
                                            version=1,
                                            primary_key=["datetime"],
                                            description="Iris flower Prediction/Outcome Monitoring"
                                            )

now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
data = {
    'prediction': [flower],
    'label': [label],
    'datetime': [now],
    }
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df, write_options={"wait_for_job" : False})

history_df = monitor_fg.read()
# Add our prediction to the history, as the history_df won't have it - 
# the insertion was done asynchronously, so it will take ~1 min to land on App
history_df = pd.concat([history_df, monitor_df])


df_recent = history_df.tail(4)
dfi.export(df_recent, './df_recent.png', table_conversion = 'matplotlib')
dataset_api.upload("./df_recent.png", "Resources/images", overwrite=True)

predictions = history_df[['prediction']]
labels = history_df[['label']]

# Only create the confusion matrix when our iris_predictions feature group has examples of all 3 iris flowers
print("Number of different flower predictions to date: " + str(predictions.value_counts().count()))
if predictions.value_counts().count() == 3:
    results = confusion_matrix(labels, predictions)

    df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                            ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])

    cm = sns.heatmap(df_cm, annot=True)
    fig = cm.get_figure()
    fig.savefig("./confusion_matrix.png")
    dataset_api.upload("./confusion_matrix.png", "Resources/images", overwrite=True)
else:
    print("You need 3 different flower predictions to create the confusion matrix.")
    print("Run the batch inference pipeline more times until you get 3 different iris flower predictions") 
